In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
import pandas as pd

from tastytrade.sessions import Credentials

from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager

from tastytrade.analytics.visualizations.custom import plot_live_candlesticks
from tastytrade.analytics.visualizations.charts import DynamicChart, Study

from datetime import datetime, timedelta

from tastytrade.analytics.studies.averages import hull

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

start_time = datetime(2025, 2, 6, 9, 30)  # datetime.now() - timedelta(minutes=30)
end_time = start_time + timedelta(hours=8)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials)

In [3]:
await dxlink.subscribe_to_candles(
    # symbol="BTC/USD:CXTALP",
    symbol="SPX",
    interval="5m",
    from_time=start_time,
    to_time=end_time,
)

In [4]:
await dxlink.subscribe_to_candles(
    # symbol="BTC/USD:CXTALP",
    symbol="SPX",
    interval="1m",
    from_time=start_time,
    to_time=end_time,
)

In [5]:
# Subscribe to 5-minute candles for SPY
# await dxlink.unsubscribe_to_candles(symbol="BTC/USD:CXTALP", interval="1m")

# await dxlink.unsubscribe_to_candles(
#     symbol="SPY",
#     interval="1m",
# )

In [6]:
# symbols = ["BTC/USD:CXTALP", "SPY", "NVDA"]
symbols = ["SPX", "NVDA"]
await dxlink.subscribe(symbols)

In [7]:
# await dxlink.unsubscribe(symbols)

In [ ]:
# Access candle data
columns = ["eventSymbol", "time", "open", "high", "low", "close"]
dxlink.router.handler[Channels.Candle].processors["feed"].df[columns].tail(10)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [ ]:
# Example usage
study_params = {"hull": {"length": 20, "price_col": "close"}}  # HMA period  # Price column to use

# Create the live chart with HMA
task = plot_live_candlesticks(dxlink=dxlink, symbol="SPX{=5m}")

In [ ]:
# Later when you want to stop
await task.cancel()

In [ ]:
task = plot_live_candlesticks(dxlink, "SPX{=5m}")

In [ ]:
task.cancel()

In [ ]:
await dxlink.close()

In [ ]:
hull(dxlink, "SPX{=5m}").head(3)

In [9]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#26A69A", "Down": "#EF5350"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)

In [ ]:
chart = DynamicChart(dxlink, "SPX{=5m}")
chart.add_study(hma_study)
chart.start()
chart.stop()

In [ ]:
chart = DynamicChart(dxlink, "SPX{=m}")
chart.add_study(hma_study)
chart.start()
chart.stop()

In [15]:
# TODOS

# Widen the plot
# Remove the scroller at the bottom
# move the legent and remove Price (that is obvious)

# Add MACD
# Add RSI
# Add Volume Profile (?? ... /ES, SPY, etc)
# Add velocity metric

# Do not plot study data points passed available data

# HULL - Align the first tick against to the subsequent tick

# ERROR if no study data found